In [189]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

In [85]:
video = cv2.VideoCapture('../project_skl/data/Subject0/10kg_top/video.mp4')

print(video.get(cv2.CAP_PROP_FRAME_COUNT))
round(video.get(cv2.CAP_PROP_FPS))

4010.0


25

In [8]:
pose_seq = np.load('../project_skl/data/Subject0/10kg_top/pose3d.npy')

In [11]:
with open('../project_skl/data/Subject0/10kg_top/pose2d.json', 'rb') as rb:
    pose2d = json.load(rb)

In [30]:
df = pd.read_csv('../project_skl/data/Subject0/10kg_top/raw_emg.csv', skiprows=60)

time_column = df.iloc[:, 0]
emg_signal = df.iloc[:, 1:]
emg_signal

,Avanti sensor 2: EMG 2 [V],Avanti sensor 4: EMG 4 [V],Avanti sensor 6: EMG 6 [V],Avanti sensor 7: EMG 7 [V],Avanti sensor 8: EMG 8 [V],Avanti sensor 10: EMG 10 [V],Avanti sensor 11: EMG 11 [V],Avanti sensor 15: EMG 15 [V]
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
347971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
347972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
347973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
347974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
def resample_emg_rms(csv_path, output_path, num_video_frames):
    # Read the CSV file
    try:
        df = pd.read_csv(csv_path)
    except:
        df = pd.read_csv(csv_path, skiprows=60)

    # Assume the first column is time or indices, and the rest are EMG channels
    time_column = df.iloc[:, 0]
    emg_signal = df.iloc[:, 1:]  # Remaining columns are EMG channels
    
    # Calculate the step size and ensure it matches the target frame count
    step_size = len(df) / num_video_frames

    # RMS downsampling with fixed window
    resampled_data = []
    for col in emg_signal.columns:
        channel_data = emg_signal[col].values
        channel_data -= channel_data.mean()
        
        rms_values = []
        for frame_idx in range(num_video_frames):
            # Calculate the window for the current frame
            center = int(frame_idx * step_size)
            start = max(0, int(center - step_size // 2))
            end = min(len(channel_data), int(center + step_size // 2))

            # Compute RMS for the window
            window = channel_data[start:end]
            rms = np.sqrt(np.mean(window ** 2))
            rms_values.append(rms)

        resampled_data.append(rms_values)

    # Generate new time indices based on video FPS
    new_time_indices = np.linspace(0, time_column.iloc[-1], num_video_frames)

    # Combine the resampled channels into a DataFrame
    resampled_df = pd.DataFrame(np.array(resampled_data).T, columns=emg_signal.columns)
    resampled_df.insert(0, 'Time', new_time_indices)

    # Save the resampled DataFrame to a new CSV file
    resampled_df.to_csv(output_path, index=False)
    print(f"Resampled EMG data saved to {output_path}")

csv_path = '../project_skl/data/Subject0/10kg_move/raw_emg.csv'
output_path = './resampled_emg_rms_fixed.csv'
# original_sampling_rate = 2.148148e+003
# video_fps = video.get(cv2.CAP_PROP_FPS)
video = cv2.VideoCapture('../project_skl/data/Subject0/10kg_move/video.mp4')
num_video_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

resample_emg_rms(csv_path, output_path, num_video_frames)

Resampled EMG data saved to ./resampled_emg_rms_fixed.csv


In [229]:
data = []

train_path = './datasplit/train'
for file in os.listdir(train_path):
    with open(os.path.join(train_path, file), 'r') as r:
        lines = r.readlines()
    
    subject_info_keys = ['age', 'height', 'weight', 'muscle', 'fat', 'arm_len', 'leg_len', 'load']
    subject_info_values = np.array([[24, 174, 70.1, 27.8, 20.3, 25.5, 39.5],
                           [23, 177, 67.0, 30.0, 13.4, 28.0, 40.0],
                           [24, 169, 69.6, 32.0, 13.5, 28.0, 40.5],
                           [23, 175, 94.0, 35.5, 31.5, 26.5, 39.5],
                           [20, 183, 86.9, 36.7, 21.8, 29.0, 42.5],
                           [25, 173, 70.0, 35.0, 15.4, 26.0, 40.0],
                           [25, 173, 77.4, 37.4, 12.6, 26.0, 42.5], 
                           [25, 169, 68.0, 29.1, 15.7, 25.0, 40.0], 
                           [24, 174, 83.2, 37.2, 18.0, 25.0, 40.0], 
                           [25, 170, 62.3, 25.3, 16.9, 26.0, 36.0],
                           [25, 180, 106.8, 49.3, 21.8, 25.0, 40.0],
                           [23, 170, 56.2, 29.5, 4.3, 25.0, 40.0],
                           [24, 173, 68.1, 32.7, 10.5, 26.0, 37.0],
                           [24, 176, 79.2, 35.6, 16.7, 26.0, 40.0],
                           [23, 176, 67.8, 30.4, 13.9, 26.0, 40.0],
                           [23, 166, 52.7, 25.4, 7.1, 24.0, 38.0]])
    
    scaler = StandardScaler()
    scaler.fit(subject_info_values)

    for _ in lines:
        x = {}
        filepath = _.strip('\n')
        x['filename'] = filepath
        x['subject'] = filepath.split('/')[2]

        x['subject_info'] = subject_info_values[0]
        weight = x['subject_info'][2]
        load = int(filepath.split('/')[3].split('kg')[0]) / weight
        x['subject_info'] = np.append(scaler.transform(subject_info_values[0].reshape(1, -1)), np.array([load]))

        x['pose3d'] = np.load(f'{filepath}/pose3d.npy')
        x['emg'] = np.load(f'{filepath}/emg_values.npy', allow_pickle=True)
        
        data.append(x)
        
data[0]['filename'], data[0]['subject'], data[0]['subject_info'], data[0]['pose3d'].shape, data[0]['emg'].shape, len(data)

('./dataset/Subject5/5kg_move/0',
 'Subject5',
 array([ 0.2       ,  0.08936332, -0.2687777 , -0.92046907,  0.72470169,
        -0.44611344, -0.13667809,  0.07132668]),
 (80, 17, 3),
 (80, 7),
 193)

In [230]:
from lightning.pytorch import loggers as pl_loggers
